# Content:

- parse IFC models into graph database
- run Diff
- generate patch
- apply patch

## Parse BIM models into graph database

In [ ]:
import time
from typing import List

from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from neo4j_middleware.neo4jConnector import Neo4jConnector

print('[INFO] Parsing Ifc StepP21 model to Neo4j.... \n')

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# paths to IFC models
model_paths: List[str] = [
    './00_sampleData/IFC_stepP21/GeomRepresentation_05/cube_single.ifc',
    './00_sampleData/IFC_stepP21/GeomRepresentation_05/cube_double.ifc']

print('[INFO] Starting to generate graphs...')
amount = len(model_paths)
start = time.perf_counter()

model_labels = []

for idx, path in enumerate(model_paths):
    # parse model
    graphGenerator = IFCGraphGenerator(connector, path, None)
    print('Generating Graph %d/%d' % (idx + 1, amount))
    label = graphGenerator.generateGraph()
    model_labels.append(label)

finish = time.perf_counter()
print('\n[INFO] 100% done. Graphs generated. Finished in {} seconds.'.format(round(finish - start, 2)))
# disconnect from database
connector.disconnect_driver()

label_init = model_labels[0]
label_updated = model_labels[1]

del idx, finish, start, model_labels, label, graphGenerator

## Run diff

runs the depth-first traversal to compare both model graphs and connects equivalent nodes using `EQUIVALENT_TO` edges

In [ ]:
from neo4jGraphDiff.GraphDiff import GraphDiff
from neo4j_middleware.ResponseParser.NodeItem import NodeItem
from neo4j_middleware.neo4jConnector import Neo4jConnector
import jsonpickle

print('Run Diff ... \n')

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# get topmost entry nodes
raw_init = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN ID(n), n.EntityType, PROPERTIES(n), LABELS(n)
    """.format(label_init))
raw_updated = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN ID(n), n.EntityType, PROPERTIES(n), LABELS(n)
    """.format(label_updated))

entry_init: NodeItem = NodeItem.from_neo4j_response_wou_rel(raw_init)[0]
entry_updated: NodeItem = NodeItem.from_neo4j_response_wou_rel(raw_updated)[0]

# run diff
pDiff = GraphDiff(connector=connector, ts_init=label_init, ts_updated=label_updated)
delta = pDiff.diff_subgraphs(entry_init, entry_updated)

# Create EQUIVALENT_TO relationships to mark all nodePairs that are matched
for p in delta.node_matching_table.matched_nodes:
    cy = """
    MATCH (n) WHERE ID(n)={0}
    MATCH (m) WHERE ID(m)= {1}
    MERGE (n)-[:EQUIVALENT_TO]->(m)
    """.format(p.init_node.id, p.updated_node.id)
    connector.run_cypher_statement(cy)

# store result
print('[INFO] saving delta ... ')
f = open('GraphDelta_init{}-updt{}.json'.format(label_init, label_updated), 'w')
f.write(jsonpickle.dumps(delta))
f.close()
print('[INFO] saving delta: DONE. ')


connector.disconnect_driver()
del raw_init, raw_updated, entry_init, entry_updated, p, GraphDiff, cy, f

## Generate patch

In [ ]:
from PatchManager.DPOService import DPOService

connector = Neo4jConnector()
connector.connect_driver()

service = DPOService(ts_init=label_init, ts_updated=label_updated)
update_patch = service.generate_DPO_patch(connector=connector)
del service

# visualize results
# update_patch.operations[0].plot_patterns()

# finally disconnect
connector.disconnect_driver()

## Apply patch

! Switch to receiver database !

In [ ]:
# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

graphGenerator = IFCGraphGenerator(connector, model_paths[0], None)
print('[INFO] Generating host graph... ' )
ts_host = graphGenerator.generateGraph()

# finally disconnect
connector.disconnect_driver()

In [ ]:
# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

update_patch.apply(ts_host)

# finally disconnect
connector.disconnect_driver()